In [2]:
from sql.methods  import get_application_by_backend_id, get_application_by_id, get_application_history, get_ubki, get_application_ids, get_ids_cmd
from ast import literal_eval
from typing  import List 
import pandas as pd 
import datetime

from utils.ubki import get_useful_ubki_fields
from utils.application_history import get_useful_history_fields
from utils.application import get_useful_application_fields

In [6]:
# Класс запроса для получения id и потом данных для датасета
class QueryMeta:
       def __init__(self, id_selection_cmd: str, application_history_info: bool, ubki_info: bool):
              self.application_ids = get_ids_cmd(id_selection_cmd)     # Загрузка id по запросу
              self.application_history_info = application_history_info # Нужна ли информация по истории
              self.ubki_info = ubki_info                               # Нужна ли информации по убки

In [7]:
# Функция создания датасета на основе QueryMeta
def create_dataset(query_metas: List[QueryMeta]) -> dict:
    result = []
    for query_meta in query_metas:                      # Проходим по объектам класса
        for app_id in query_meta.application_ids:       # Проходим по id конкретного объекта
            # Получаем анкету
            app_info = get_application_by_id(app_id)
            # Получаем историю анкет
            app_history_info = get_application_history(app_info['user_id'], app_info['applied_at']) if query_meta.application_history_info else {} 
            # Получаем убки
            try:
                ubki_info = get_ubki(app_info['social_number'], app_info['id']) if query_meta.ubki_info else {}
            except:
                ubki_info = {}
            # Добавляем в результат одного клиента с полезными данными по каждому пункту инф.   ## !! Можно расширить получаемые полезные данные с каждого елемента !!
            result.append({ **get_useful_application_fields(app_info),
                            **get_useful_history_fields(app_history_info),
                            **get_useful_ubki_fields(ubki_info, app_info['phone_mobile'], app_info['email'], app_info['applied_at'])})
    return result

# ТЕСТИРОВАНИЕ

In [8]:
DATE_FROM = datetime.datetime(year=2020, month=1, day=1)
DATE_TO = datetime.datetime(year=2021, month=5, day=1)

In [9]:
q = []
for i in range(10):
    cmd =   """ SELECT id FROM finplugs_pdl_applications
                WHERE (applied_at between '{date_from}' AND '{date_to}') AND status_id IN (5,6,2) 
                ORDER BY RAND() 
                LIMIT {limit}
            """.format(date_from=DATE_FROM, date_to=DATE_TO, limit=2*i)
    q.append(QueryMeta(cmd, True, True))

In [11]:
for iter, val in zip(range(len(q)), q):
    print(' i = {id}, ids = {ids}'.format(id=i, ids=val.application_ids))

 i = 9, ids = []
 i = 9, ids = [1101716, 872142]
 i = 9, ids = [806439, 895522, 843565, 1055130]
 i = 9, ids = [1049036, 1060896, 803303, 945011, 790715, 1026189]
 i = 9, ids = [1122899, 1114490, 1054246, 872260, 790740, 1127960, 1114132, 887589]
 i = 9, ids = [831484, 919154, 1030251, 796914, 1022390, 804008, 797103, 965632, 1041429, 953087]
 i = 9, ids = [838124, 1086290, 964384, 974566, 852079, 1025968, 1131049, 891644, 889452, 805082, 767535, 998601]
 i = 9, ids = [892145, 1013658, 906722, 851833, 949852, 1080502, 1003375, 978610, 841789, 904240, 1097800, 786165, 918066, 811749]
 i = 9, ids = [812148, 1089602, 921530, 767921, 1039512, 976964, 962059, 916529, 1040096, 883049, 1024929, 799123, 888054, 764374, 836828, 843536]
 i = 9, ids = [1064981, 1084949, 903597, 839468, 897965, 1068036, 1123565, 850783, 1027224, 960294, 864046, 812022, 957021, 1034208, 1010398, 806152, 1088190, 1117805]


In [ ]:
dataset = create_dataset(q)
dataset

In [13]:
dataset = pd.DataFrame(dataset)

In [14]:
dataset.to_excel('check.xlsx', index = False)